# Testing methods of Mishra et al. 2022

[link to paper](http://journals.sagepub.com/doi/10.1177/0272989X211044697)

I will use the Behavioral Risk Factor Surveillance System data set from the CDC. I have 5 datasets from 2011-2015 available on Kaggle.  I will probably start by dividing by race or gender unevenly in the train and recalibration sets and go from there.

Measure of clinical utility is the standardized net benefit.

* $ R \equiv $ the risk threshold for recommending an intervention.
* $ \pi $ is the prevalence of the outcome
* $\text{TPR}_R \equiv$ the true positive rate when the model risk threshold is $R$. AKA the sensitivity or the hit rate.
* $\text{FPR}_R \equiv$ the same for the false positive rate. AKA 1 - specificity or the probability of a false alarm.
* The ratio $R/(1-R)$ represents the ratio of benefits and harms of recommending an intervention. In this case I would assume this means the benefits vs. harms of screening a patient.

$$ \text{sNB}_R = \text{TPR}_R - \frac{R}{1-R}\frac{1-\pi}{\pi} \text{FPR}_R$$

The advantage of this measure of clinical utility is that its maximum is 1 when the TPR=1 and FPR=0. It is an "opt-in" formulation that assumes the default should be not treating or screening the patient. Other measures of clinical utility include the net benefit and relative utility.

The behavior of the sNB is that as the risk becomes large, it penalizes a higher false positive rate. As the prevalence becomes higher, it penalizes false positives less. For the rare-disease case, we might expect that the risk is high of non-detection, but that the prevalence is somewhat low. So perhaps we want the threshold $R$ closer to 0 to indicate that the our threshold for screening is low.

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

## Data set 
* This dataset can be downloaded from [link](https://www.kaggle.com/datasets/cdc/behavioral-risk-factor-surveillance-system?select=2015.csv)
* The column information can be found at [link](https://www.cdc.gov/brfss/annual_data/2015/pdf/codebook15_llcp.pdf)
* List of columns an

In [37]:
datafile = '~/Documents/datasets/LLCP2015.XPT' # update with your own path!

In [38]:
data = pd.read_sas(datafile)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sarahcwesolowski/Documents/datasets/LLCP2015.XPT'

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 441456 entries, 0 to 441455
Columns: 330 entries, _STATE to _AIDTST3
dtypes: float64(323), object(7)
memory usage: 1.1+ GB


### Encode state groups to regions

Use one-hot encoding to indicate a region. Then we can calibrate the model in the region, using another region for recalibration and testing. These can certainly be grouped differently, but I am grouping them like this for testing purposes only.

Could also group subjects randomly.

In [25]:
southeast_states = [1, 5, 10, 11, 12, 13, 21, 22, 24, 28, 29, 37, 45, 47, 51, 54]
northeast_states = [9, 23, 25, 33, 34, 36, 42, 44, 50]
midwest_states = [17, 18, 19, 20, 26, 27, 31, 38, 39, 46, 55]
southwest_states = [4, 40, 35, 48]
west_states = [6, 8, 16, 30, 32, 41, 49, 53, 56]
other = [2, 15, 66, 72] # Alaska, Hawaii, Guam, PR

In [30]:
state_cond = [
    data['_STATE'].isin(southeast_states),
    data['_STATE'].isin(northeast_states),
    data['_STATE'].isin(midwest_states),
    data['_STATE'].isin(southwest_states),
    data['_STATE'].isin(west_states),
    data['_STATE'].isin(other)
    ]
state_codes = ['SE', 'NE', 'MW', 'SW', 'W', 'OTHER']
data['REGION'] = np.select(state_cond, state_codes)

In [31]:
data['REGION'].value_counts()

MW       114386
SE       114036
NE        79535
W         79285
SW        36320
OTHER     17894
Name: REGION, dtype: int64

### Categorize the columns

In [33]:
for c in data.columns:
    print(c)

_STATE
FMONTH
IDATE
IMONTH
IDAY
IYEAR
DISPCODE
SEQNO
_PSU
CTELENUM
PVTRESD1
COLGHOUS
STATERES
CELLFON3
LADULT
NUMADULT
NUMMEN
NUMWOMEN
CTELNUM1
CELLFON2
CADULT
PVTRESD2
CCLGHOUS
CSTATE
LANDLINE
HHADULT
GENHLTH
PHYSHLTH
MENTHLTH
POORHLTH
HLTHPLN1
PERSDOC2
MEDCOST
CHECKUP1
BPHIGH4
BPMEDS
BLOODCHO
CHOLCHK
TOLDHI2
CVDINFR4
CVDCRHD4
CVDSTRK3
ASTHMA3
ASTHNOW
CHCSCNCR
CHCOCNCR
CHCCOPD1
HAVARTH3
ADDEPEV2
CHCKIDNY
DIABETE3
DIABAGE2
SEX
MARITAL
EDUCA
RENTHOM1
NUMHHOL2
NUMPHON2
CPDEMO1
VETERAN3
EMPLOY1
CHILDREN
INCOME2
INTERNET
WEIGHT2
HEIGHT3
PREGNANT
QLACTLM2
USEEQUIP
BLIND
DECIDE
DIFFWALK
DIFFDRES
DIFFALON
SMOKE100
SMOKDAY2
STOPSMK2
LASTSMK2
USENOW3
ALCDAY5
AVEDRNK2
DRNK3GE5
MAXDRNKS
FRUITJU1
FRUIT1
FVBEANS
FVGREEN
FVORANG
VEGETAB1
EXERANY2
EXRACT11
EXEROFT1
EXERHMM1
EXRACT21
EXEROFT2
EXERHMM2
STRENGTH
LMTJOIN3
ARTHDIS2
ARTHSOCL
JOINPAIN
SEATBELT
FLUSHOT6
FLSHTMY2
IMFVPLAC
PNEUVAC3
HIVTST6
HIVTSTD3
WHRTST10
PDIABTST
PREDIAB1
INSULIN
BLDSUGAR
FEETCHK2
DOCTDIAB
CHKHEMO3
FEETCHK
EYEEXAM
DIABEYE
D

In [ ]:
# section 0 record identification
# 'DISPCODE' : whether interview was completed
# 
sec0 = ['_STATE', 'FMONTH', 'IDATE', 'IMONTH',\
        'IYEAR', 'DISPCODE', 'SEQNO', '_PSU',\
        'CTELENUM', 'PVTRESD1', 'COLHOUS', 'STATERES',\
        'CELLFON3', 'LADULT', 'NUMADULT', 'NUMMEN',\
        'NUMWOMEN', 'CTELNUM1', 'CELLFON2', 'CADULT',\
        'PVTRESD2', 'CCLGHOUS', 'CSTATE', 'LANDLINE',\
        'HHADULT']
# section 1/2 general health
# how would you say is your general health, #days health not good, #days mental health not good,
# poor helath keep you from regular activities
general_health = ['GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH']
# section 3 health care access
# do you have health care coverage of any kind, do you have a pcp, couldn ot see dr bc of cost,
# how long since last checkup
healthcare_access = ['HLTHPLN1', 'PERSDOC2', 'MEDCOST', 'CHECKUP1']
# section 4/5 htn awareness, cholestorol awareness
# 'BPHIGH4' : ever been told you have high bp, possibly only during pregnancy
# 'BPMEDS' : are you currently on medication for high bp
# 'BLOODCHO' : have ever had cholesterol checked?
# 'CHOLCHK' : how long since it was checked
# 'TOLDHI2' : ever been told blood cholesterol is high
awareness = ['BPHIGH4', 'BPMEDS', 'BLOODCHO', 'CHOLCHK',
            'TOLDHI2']
# Section 6
# heart attack, heart disease, stroke, asthma, currently have asthma, skin cancer, other cancer
# copd and related, arthritis diseases, depressive disorders, kidney diseases, diabetes,
# if diabetes, what age
chronic_health_cond = ['CVDINFR4', 'CVDCRHD4', 'CVDSTRK3', 'ASTHMA3',\
                       'ASTHNOW', 'CHCSCNCR', 'CHCOCNCR', 'CHCCOPD1',\
                       'HAVARTH3', 'ADDEPEV2', 'CHCKIDNY', 'DIABETE3',\
                       'DIABAGE2']
# Section 7 demographics
# sex, marital status, education, rent vs own, more than phone number in household, how many are residential,
# do you have a cell phone, are you a veteran, employment status, number of children in household,
# annual income, use the internet, weight, height, pregnant?, physical limitations, special equipment,
# blindness/difficulty seeing, difficulty concentrating, problems walking, difficulty dressing/bathing,
# difficulty with errands
demographics = ['SEX', 'MARITAL', 'EDUCA', 'RENTHOM1',\
                'NUMHHOL2', 'NUMPHON2', 'CPDEMO1', 'VETERAN3',\
                'EMPLOY1', 'CHILDREN', 'INCOME2', 'INTERNET',\
                'WEIGHT2', 'HEIGHT3', 'PREGNANT', 'QLACTLM2',\
                'USEEQUIP', 'BLIND', 'DECIDE', 'DIFFWALK',
                'DIFFDRES', 'DIFFALON']
# section 8 tobacco usage
# have you smoked 100 cigarettes in entire life, smoke how many days, trying to quit, how long since last smoked
# chewing tobacco, 
tobacco = ['SMOKE100', 'SMOKDAY2', 'STOPSMK2', 'LASTSMK2',\
          'USENOW3']
# section 9 alcohol
# how many days per week or month did you have alcohol, during past 30 days how many drinks per day on average,
# how many days over 5/4 drinks for men and women, largest # of drinks in past 30 days
# 
alcohol = ['ALCDAY5', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS']
# section 10 fruits and vegetables
# how many time per day/week/month did you drink 100% juice, how much did you eat fruit, beans,
# dark green vegetables, orange vegetables, any other vegetables, 
f_and_v = ['FRUITJU1', 'FRUIT1', 'FVBEANS', 'FVGREEN',\
           'FVORANG', 'VEGETAB1']
# section 11 exercise
# participate in any physical activity other than job, what type of exercise (some other ones I will drop)
# strength training activity, 
exercise = ['EXERANY2', 'EXRACT11', 'EXEROFT1', 'EXERHMM1',\
            'EXERACT21', 'EXEROFT2', 'EXERHMM2', 'STRENGTH']

In [ ]:

drop = ['FMONTH', 'IDATE', 'IMONTH', 'IYEAR', ]

### Drop columns with too many missing fields